In [1]:
import numpy as np
import matplotlib.pyplot as plt

1. Измените функцию calc_logloss так, чтобы нули по возможности не попадали в np.log (как вариант - использовать np.clip или np.where).

In [2]:
def calc_logloss(y, p):
    np.clip(p, a_min=1e-16, a_max=(1-1e-16), out=p)
    err = np.mean(- y * np.log(p) - (1.0 - y) * np.log(1.0 - p))
    return err

In [3]:
y = np.array([0, 0, 1, 0, 1, 0, 1, 0, 1, 1])
p = np.array([0.26, 0.31, 0.82, 0.28, 0.79, 0.57, 0.94, 0.15, 0.25, 0.86])
calc_logloss(y, p)

0.40403277677565635

2. На данных из урока изучите влияние гиперпараметров на ошибку алгоритма. Подберите аргументы функции eval_LR_model для логистической регрессии таким образом, чтобы log loss не превышал значение 0.3 (0.1). Как изменились веса?

In [4]:
X = np.array([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
              [1, 1, 2, 1, 3, 0, 5, 10, 1, 2],  # стаж репетитора
              [500, 700, 750, 600, 1450,        # средняя стоимость занятия
               800, 1500, 2000, 450, 1000],
              [1, 1, 2, 1, 2, 1, 3, 3, 1, 2]],  # квалификация репетитора
              dtype = np.float64).T 

y = np.array([0, 0, 1, 0, 1, 0, 1, 0, 1, 1]) # поступил или нет ученик на специальность Математика

In [5]:
def standardization(x: np.ndarray) -> np.ndarray:
    s = (x - x.mean()) / x.std()
    return s

X_st = X.copy()
X_st[:, 2] = standardization(X[:, 2])
X_st[:, 2]

array([-0.97958969, -0.56713087, -0.46401617, -0.77336028,  0.97958969,
       -0.36090146,  1.08270439,  2.11385144, -1.08270439,  0.05155735])

In [6]:
# логистическая регрессия
def eval_LR_model(X, y, iterations, eta=1e-4):
    np.random.seed(42)
    w = np.random.randn(X.shape[1])
    n = X.shape[0]
    for i in range(1, iterations + 1):
        z = np.dot(X, w) # log(p/(1-p))
        pred = sigmoid(z) # p [0, 1]
        err = calc_logloss(y, pred)
        w -= eta * (1/n * np.dot((pred - y), X))
        if i % (iterations / 10) == 0:
            print(i, w, err)
    return w

def sigmoid(z):
    return 1 / (1 + np.exp(-z))

In [7]:
w = eval_LR_model(X_st, y, 40000, 1e-1)

4000 [-6.96133333 -1.14371437 -0.98694403  6.13057114] 0.3066241878369147
8000 [-10.04075209  -1.35657036  -1.99546097   8.51208184] 0.26560192980297875
12000 [-12.39769211  -1.54494868  -2.73269053  10.39838241] 0.24130563028249802
16000 [-14.4278092   -1.71463311  -3.35508641  12.03835138] 0.22321425525097446
20000 [-16.2427091   -1.86940882  -3.90475531  13.50901763] 0.20874421725365058
24000 [-17.89249952  -2.01170597  -4.40004665  14.84732364] 0.1967896979025623
28000 [-19.40725405  -2.1433189   -4.85167146  16.07643552] 0.18671757983422052
32000 [-20.80815977  -2.26566925  -5.26699378  17.21308546] 0.17810833911174004
36000 [-22.1114095   -2.37992046  -5.65152931  18.27023302] 0.17066275600778646
40000 [-23.32991383  -2.48704062  -6.00960348  19.2582858 ] 0.16415855743912566


Ответ: чтобы получить ошибку меньше, нужно увеличивать количество итераций

3. Создайте функцию calc_pred_proba, возвращающую предсказанную вероятность класса "1". На вход функции подаются значения признаков Х и веса, которые уже посчитаны функцией eval_LR_model.

In [8]:
def calc_pred_proba(X, w):
    return sigmoid(np.dot(X, w))

In [9]:
def eval_LR_model(X, y, iterations, eta=1e-4):
    np.random.seed(42)
    w = np.random.randn(X.shape[1])
    n = X.shape[0]
    for i in range(1, iterations + 1):
        z = np.dot(X, w) # log(p/(1-p))
        pred = sigmoid(z) # p [0, 1]
        err = calc_logloss(y, pred)
        w -= eta * (1/n * np.dot((pred - y), X))
        if i % (iterations / 15) == 0:
            print(i, w, err)
    return w

In [10]:
w = eval_LR_model(X_st, y, 300, 1e-1)
pred = calc_pred_proba(X_st, w)
pred


20 [ 0.14594713 -0.69347808  0.78221643  1.2114552 ] 0.5697629147350483
40 [-0.01086542 -0.68365659  0.90991129  1.22344049] 0.5480608521548667
60 [-0.12559098 -0.69176936  0.98552719  1.26867802] 0.5372341719773464
80 [-0.21797982 -0.70784853  1.03250384  1.32962077] 0.5297726356017476
100 [-0.29848458 -0.72686866  1.06314016  1.39690737] 0.523576748599366
120 [-0.37249361 -0.74644348  1.08384568  1.46580885] 0.5180355499063359
140 [-0.44277373 -0.76546532  1.0980349   1.53403769] 0.5129405286552332
160 [-0.5107546  -0.78345157  1.10759523  1.60054865] 0.5081975348124199
180 [-0.57718376 -0.80022524  1.11362445  1.66491815] 0.5037496014005538
200 [-0.64245884 -0.81575707  1.11680356  1.72702881] 0.4995552177931558
220 [-0.70679654 -0.830087    1.11758722  1.7869102 ] 0.4955814666185855
240 [-0.77031908 -0.84328505  1.11630244  1.84465873] 0.4918013359785121
260 [-0.83309869 -0.85543189  1.11320054  1.9003976 ] 0.48819232810693763
280 [-0.89518081 -0.86660948  1.10848522  1.95425707] 0

array([0.28762292, 0.38881594, 0.68801221, 0.3363451 , 0.81835964,
       0.65744682, 0.86663304, 0.20160237, 0.26490601, 0.79562537])

Ответ: остановка на 280 итерации

4. Создайте функцию calc_pred, возвращающую предсказанные классы (0 или 1). На вход функции подаются значения признаков Х и веса, которые уже посчитаны функцией eval_LR_model, а также порог вероятности.

In [12]:
def calc_pred(X, w):
    return np.round(calc_pred_proba(X, w), 0)

In [13]:
y_pred = calc_pred(X_st, w)
print('y_pred:',y_pred)
print('y:     ',y)

y_pred: [0. 0. 1. 0. 1. 1. 1. 0. 0. 1.]
y:      [0 0 1 0 1 0 1 0 1 1]
